In [ ]:
import opensrane as opr
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from copy import deepcopy

## Gas dispersion Using Guassian model

In [ ]:
#Verification by casal example 2.9
opr.wipe()

SiteTAg=1
opr.Sites.Site(SiteTAg, Temperature=20+273,Pressure=1.0132*10**5 ,XSiteBoundary=[0,100,100,0], YSiteBoundary=[0,0,100,100], g=9.81)


#Create Wind Object
windobj=opr.WindData.WindRose(tag=1)
windobj.WindClass="A"
windobj.WindDirection=90
windobj.WindSpeed=3
windobj.AlphaCOEF=1
windobj.isCalmn=False


#Define Material
subsObj=opr.Substance.DataBank.CasCalEx2_1(1)
subsObj.Specific_Heat_of_Vaporization=392.23*1000
subsObj.Vapour_Pressure=16130                      #Accoring Casal Example 2.9
subsObj.Molecular_Weight=86/1000                   #Accoring Casal Example 2.9
subsObj.Boiling_Point=273+68.7                     #According Casal Example 2.9
subsObj.Liquid_Partial_Pressure_in_Atmosphere=0    #According Casal Example 2.9
subsObj.Lower_Flammability_Limit=0.03
subsObj.Upper_Flammability_Limit=3

#Define Outflow Models
tag=1
opr.OutFlowModel.SimultaniousGas(tag, Release_Ratio=0.001)

opr.Safety.Dike(tag=1, Height=10, Area=3.1415*11**2)

#Define Plant Units
xc=0
yc=0
D=5
#OnGroundTag=1
Uni=opr.PlantUnits.ONGStorage(1,SiteTag=SiteTAg,DikeTag=1,
                              Horizontal_localPosition=xc,Vertical_localPosition=yc,
                              Pressure=1.5*10**5 ,Temperature=2,
                              Diameter=5,Height=10,
                              SubstanceTag=1,SubstanceVolumeRatio=0.002) #Define a StorageTank

Uni.isdamaged=True

#These steps will be done Automatically By the Program inside the Analysis Part
OutFlowObj=opr.OutFlowModel.ObjManager[1]     # Get OutFlow Object
UnitObj=opr.PlantUnits.ObjManager[1]          # Get Unit Object
Subs=opr.Substance.ObjManager[1]              # Get Substance Object
SiteObj=opr.Sites.ObjManager[1]               # Get Site Object
UnitObj.OutFlowModelObject=deepcopy(OutFlowObj)  #Assign OutFlow Model to Unit Object
UnitObj.OutFlowModelObject.UnitObject=UnitObj
UnitObj.OutFlowModelObject.Calculate(); #Calculate OutFlow Calculations to get 


#Define Dispesion Spread Models and their connections to the materials and outflows
DispObject=opr.DispersionSpreadModels.GasGaussian(tag=1,MatTags=[1], OutFlowModelTags=[1,2,3,4]) #Creat the Dispersion Object
UnitObj.DispersionSpreadModelObject=DispObject           #Assign Dispersion Object to the Unit Object
UnitObj.DispersionSpreadModelObject.UnitObject=UnitObj   #Assign the UnitObject to the Dispersion Model
UnitObj.DispersionSpreadModelname=DispObject.name

#Do Calculations
DispObject.Calculate();


x=[i/10 for i in range(-100,110)]
C=[DispObject.GasConcentration(4,X,0.5) for X in x]

plt.figure()
plt.plot(x,C)
plt.ylabel('C')
plt.xlabel('y')
plt.title('Concentration at x=4 and z=0.5')

In [ ]:
#Plot the sampled plant unit
opr.Plot.Plotly.PlotUnits2D(GasConcentrationlist=[0.1,0.3],GasConcentrationHeght=0.5)

In [ ]:
#Plot cross section of the plume Concentration values and longitudinal Concentration values
x=[i/10 for i in range(-100,110)]
C=[DispObject.GasConcentration(4,X,0.5) for X in x]

plt.figure()
plt.plot(x,C)
plt.ylabel('C')
plt.xlabel('y')
plt.title('Concentration at x=4 and z=0.5')


z=0.5
x=[i/100 for i in range(-100,600)]
C=[DispObject.GasConcentration(X,0,z) for X in x]





plt.figure()
plt.plot(x,C)
plt.ylabel('C')
plt.xlabel('x')
plt.title('Concentration at y=0 and z=0.5')


z=1
x=[i/100 for i in range(-10,50)]
C=[DispObject.GasConcentration(X,0,z) for X in x]





plt.figure()
plt.plot(x,C)
plt.ylabel('C')
plt.xlabel('x')
plt.title('Concentration at y=0 and z=1')
# print(list(zip(x,C)))
x,z=DispObject._FindZXCMax()
print(DispObject._FindZXCMax())
print(DispObject._LocalGasConcentration(x,0,z))

In [ ]:
#Another Example to show the centyer line and dispersed gas and ...
#Calculate center line

LFL=0.03
UFL=3

x,z,zt,yl,xpoints,ypoints,zpoints,Dx,Dy,Dz,Conc=DispObject._GasPoints(LFL,UFL,SegmentNumbers=10,errpercent=1)

plt.figure()
plt.plot(x,z)
plt.ylabel('z')
plt.xlabel('x')
plt.title('Center Line of Concentration plume')

In [ ]:
# Plot 3D shape of the releases particles in the range of the LFL and UFL 
m=max(Conc)

color=[f'rgba(250,0,0,{int(i/m*1000)/1000})' for i in Conc]

fig=go.Figure()

fig.add_scatter3d(x=xpoints,y=ypoints,z=zpoints,marker=dict(color=color,size=3),mode='markers')
fig.update_layout(width=1000,height=800)
fig.show()

In [ ]:
#Calculate the Mass and Center mass of the released Gass
print(DispObject._MassCMass(Dx,Dy,Dz,xpoints,zpoints,Concentrations=Conc))


In [ ]:
#It is show how to show simply the gas points BUT this is initial and above codes are very fast
DispObject.GasConcentration(1,0,1)



x=[i for i in range(0,25)]
y=[i for i in range(-14,14)]
z=[i/3 for i in range(0,15)]

boundx=[]
boundy=[]
boundz=[]



SpacePoints=[]
color=[]
for i in x:
    for j in y:
        for k in z:
            SpacePoints.append((i,j,k))
            boundx.append(i)
            boundy.append(j)
            boundz.append(k)
            
            
color=[DispObject.GasConcentration(i[0],i[1],i[2]) for i in SpacePoints]            
m=max(color)

color=[f'rgba(0,250,0,{int(i/m*100)/100})' for i in color]


fig=go.Figure()



fig.add_scatter3d(x=boundx,y=boundy,z=boundz,marker=dict(color=color,size=3),mode='markers')
fig.update_layout(width=1000,height=800)
fig.show()

In [ ]:
# Usage of Give Boundary
points=DispObject.GiveBoundary(C=0.1,z=0.5,SegmentNumbers=10,errpercent=1)
# print(points)

In [ ]:
# Plot of the Given Boundary
import plotly.graph_objects as go

fig=go.Figure()

fig.add_scatter(x=[p[0] for p in points], y=[p[1] for p in points],mode='lines',line=dict(color=f'rgba(250, 250, 5,1)'),fill='toself') #'['none', 'tozeroy', 'tozerox', 'tonexty', 'tonextx',  'toself', 'tonext']'
          